## Bikespace Analysis - Damaged Bicycle Parking Reports

This notebook takes user-submitted reports of damaged bicycle parking from the BikeSpace app and returns the nearest 5 or fewer City of Toronto bicycle parking features based on geographic proximity. The goal is to identify City bicycle parking that may need to be replaced or repaired.


### TODO

* Improve excel format for data tabs and reorder columns
* Incorporate survey notes

In [73]:
# imports
import datetime
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gpd
import requests


### Get Data - Bikespace Reports

The [BikeSpace app](https://bikespace.ca/) allows users to report issues with bicycle parking in Toronto, including parking features that are damaged. User reports can be viewed on the [BikeSpace dashboard](https://dashboard.bikespace.ca/) or downloaded via the API.

Details on the bikespace API can be found at [api-dev.bikespace.ca](https://api-dev.bikespace.ca/api/v2/docs).


In [2]:
# get bikespace reports
report_limit = 5000
bikespace_request = requests.get(
  "https://api-dev.bikespace.ca/api/v2/submissions",
  params={"limit": report_limit})
bikespace_response = json.loads(bikespace_request.text)
bikespace_reports_data = pd.DataFrame(bikespace_response['submissions']).set_index('id')

In [3]:
# convert to geodataframe
bikespace_reports = gpd.GeoDataFrame(bikespace_reports_data, 
  geometry=gpd.points_from_xy(
    bikespace_reports_data['longitude'], 
    bikespace_reports_data['latitude'],
    ),
  crs="EPSG:4326"
  ) #.drop(["latitude", "longitude"], axis=1)

bikespace_reports['issues'].explode().value_counts()

issues
not_provided    512
damaged         400
full            359
other           158
abandoned        16
Name: count, dtype: int64

In [4]:
# get toronto ward boundaries
# https://open.toronto.ca/dataset/city-wards/
toronto_wards = gpd.read_file("https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/5e7a8234-f805-43ac-820f-03d7c360b588/resource/737b29e0-8329-4260-b6af-21555ab24f28/download/City%20Wards%20Data.geojson")

In [5]:
# bikespace reports within Toronto only
br_toronto = bikespace_reports.sjoin(toronto_wards[["geometry", "AREA_DESC"]], how="inner", predicate="intersects").drop("index_right", axis=1)

### Sources of City Bicycle Parking Data

#### Option A

Source datasets from [open.toronto.ca](https://open.toronto.ca/)

#### Option B

Uses "normalized" city bicycle parking data from [github.com/tallcoleman/new-parking-map/](https://github.com/tallcoleman/new-parking-map/). This data makes the data fields consistent for all datasets (based on the OpenStreetMap tagging system) and filters out bicycle parking features for which the data indicates they are not current present on the street.

Relevant filters applied:

* bicycle-parking-racks: STATUS must be "Installed". ("Delivered", "Approved", "Proposed", and "TBD" features are not included)
* street-furniture-bicycle-parking: STATUS must be "Existing". ("Temporarily Removed" features are not included)

In [6]:
# read source urls and other metadata from open_toronto_ca_sources.json
city_sources_path = Path("open_toronto_ca_sources.json")
with city_sources_path.open("r") as f:
  city_sources = json.load(f)


In [7]:
# get city bicycle parking data - OPTION A
city_data = {}
for source in city_sources['datasets']:
  city_data[source['dataset_name']] = gpd.read_file(source['download_url'])
  city_data[source['dataset_name']].insert(0, "source", source['dataset_name'])

city_data_all = pd.concat(city_data.values())


In [8]:
# get city bicycle parking data - OPTION B
normalized_data = {}
for source in city_sources['datasets']:
  normalized_data[source['dataset_name']] = gpd.read_file(source['normalized_url'])

normalized_data_all = pd.concat(normalized_data.values())

In [9]:
# convert datetime values to string
for column in normalized_data_all:
  if "datetime" in str(normalized_data_all[column].dtype):
    normalized_data_all[column] = normalized_data_all[column].map(
      lambda x: x.isoformat()
    )

### Damage reports and closest parking feature

In [10]:
br_toronto_damaged = br_toronto[["damaged" in i for i in br_toronto['issues']]]

In [11]:
# convert crs to allow for distance calculations in metres
br_toronto_damaged_utm17n = br_toronto_damaged.to_crs("32617")
city_data_all_utm17n = city_data_all.to_crs("32617")


In [12]:
# area to search, in metres
search_radius = 30

# option A - nearest join only
nearest_features_damaged = br_toronto_damaged_utm17n.sjoin_nearest(
  city_data_all_utm17n.assign(
    city_geometry=city_data_all_utm17n['geometry']
    ), 
  how="inner", 
  max_distance=search_radius, 
  distance_col="distance"
  )
nearest_features_damaged.columns

Index(['comments', 'issues', 'latitude', 'longitude', 'parking_duration',
       'parking_time', 'geometry', 'AREA_DESC', 'index_right', 'source', '_id',
       'ADDRESS_POINT_ID', 'ADDRESS_NUMBER', 'LINEAR_NAME_FULL',
       'ADDRESS_FULL', 'POSTAL_CODE', 'MUNICIPALITY', 'CITY', 'WARD',
       'PLACE_NAME', 'GENERAL_USE_CODE', 'CENTRELINE_ID', 'LO_NUM',
       'LO_NUM_SUF', 'HI_NUM', 'HI_NUM_SUF', 'LINEAR_NAME_ID', 'ID',
       'PARKING_TYPE', 'FLANKING', 'BICYCLE_CAPACITY', 'SIZE_M',
       'YEAR_INSTALLED', 'BY_LAW', 'DETAILS', 'OBJECTID', 'WARD_NAME',
       'MI_PRINX', 'CAPACITY', 'MULTIMODAL', 'SEASONAL', 'SHELTERED',
       'SURFACE', 'STATUS', 'LOCATION', 'NOTES', 'MAP_CLASS',
       'ADDRESSNUMBERTEXT', 'ADDRESSSTREET', 'FRONTINGSTREET', 'SIDE',
       'FROMSTREET', 'DIRECTION', 'SITEID', 'BIA', 'ASSETTYPE', 'SDE_STATE_ID',
       'city_geometry', 'distance'],
      dtype='object')

#### Try using buffer instead

In [13]:
br_toronto_damaged_utm17n = br_toronto_damaged_utm17n.assign(
  geometry_buffered = br_toronto_damaged_utm17n.buffer(search_radius)
)

data_matches = city_data_all_utm17n.sjoin(
  df=br_toronto_damaged_utm17n[["geometry_buffered"]].set_geometry("geometry_buffered"),
  how='inner',
  predicate='intersects'
)

data_matches.sample(5)

,source,_id,ADDRESS_POINT_ID,ADDRESS_NUMBER,LINEAR_NAME_FULL,ADDRESS_FULL,POSTAL_CODE,MUNICIPALITY,CITY,WARD,...,ADDRESSSTREET,FRONTINGSTREET,SIDE,FROMSTREET,DIRECTION,SITEID,BIA,ASSETTYPE,SDE_STATE_ID,index_right
62,bicycle-parking-racks,63,8417835.0,1,Albany Ave,1 Albany Ave,M5R 3C2,former TORONTO,Toronto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,688
7646,street-furniture-bicycle-parking,7647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04,...,Bloor St W,Bloor St W,South,Indian Rd,East,,Bloor By The Park,Ring,0.0,1158
16799,street-furniture-bicycle-parking,16800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,...,Dupont St,Dupont St,North,Hammond Pl,East,,,Ring,0.0,979
9398,street-furniture-bicycle-parking,9399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,...,Dundas St W,Dundas St W,North,Claremont St,East,,Trinity-Bellwoods,Ring,0.0,740
3361,street-furniture-bicycle-parking,3362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,...,Front St E,Rolling Mills Rd,West,Front St E,South,,,Ring,0.0,403


In [95]:
# gpd.GeoDataFrame([br_toronto_damaged_utm17n.loc[i] for i in data_matches['index_right']])

report_matches = gpd.GeoDataFrame(
  [
    br_toronto_damaged_utm17n.loc[i] 
    for i 
    in data_matches['index_right']
  ],
  crs="32617"
)

distances = data_matches['geometry'].distance(report_matches, align=False)
data_matches = data_matches.assign(distance=distances)
# reorder columns
data_matches = data_matches[
  ['distance'] + [col for col in data_matches.columns if col != 'distance']
]
data_matches = (data_matches
  .to_crs(4326)
  .explode(index_parts=False) # convert multipoint to point
  .assign(
    latitude=lambda r: [y for y in r.geometry.y],
    longitude=lambda r: [x for x in r.geometry.x]
  )
)

In [96]:
report_matches_unique = (report_matches
  [~report_matches.index.duplicated(keep='first')]
  .drop(columns=["geometry_buffered"])
  .sort_index(ascending=False)
  .assign(url=lambda x: [f"https://dashboard.bikespace.ca/#feed?view_all=1&submission_id={id}" for id in x.index])
  .to_crs(4326)
)

In [97]:
# organize BikeSpace damage reports alongside top 5 nearest city parking features
report_city_matches = []

for ix in report_matches_unique.index:
  report_city_matches.append({
    "report": report_matches_unique.loc[[ix]],
    "city_features": data_matches[
      data_matches['index_right'] == ix
      ].nsmallest(n=5, columns="distance"),
  })



In [98]:
matched_city_features_unique = (
  pd.concat(
    [df["city_features"] for df in report_city_matches]
  )
)

In [100]:
# output

# set up output excel sheet
writer = pd.ExcelWriter(
  'damage_bikespace_city_matches.xlsx', 
  engine='xlsxwriter',
)
workbook = writer.book
bold = workbook.add_format({'bold': True})


# TAB 1 - DISPLAY OF REPORTS AND MATCHES
worksheet = workbook.add_worksheet('Matches')
writer.sheets['Matches'] = worksheet

# write header content
worksheet.write(
  'A1', 
  "Bikespace Analysis - Damaged Bicycle Parking Reports", 
  bold,
)
worksheet.write(
  'A2',
  f"Updated {datetime.datetime.today().strftime('%B %d %Y')}",
)
worksheet.write(
  'A3',
  f"{len(report_city_matches)} BikeSpace damage reports with nearby City bicycle parking features"
)

# write data tables
write_row = 4
for pair in report_city_matches:
  report, city_features = pair.values()
  report = (report
    .reset_index(names=["id"])
    .drop(columns=["geometry"])
    .T
  )
  city_features = (city_features
    .drop(columns=["index_right", "geometry"])
    .T
    .replace(0, np.nan)
    .replace("", np.nan)
    .dropna()
  )
  report.to_excel(
    writer, 
    sheet_name='Matches', 
    startrow=write_row, 
    startcol=0,
    header=False,
  )
  write_row += len(report) + 1
  city_features.to_excel(
    writer, 
    sheet_name='Matches', 
    startrow=write_row, 
    startcol=0,
    header=False,
  )
  write_row += len(city_features) + 2

# formatting
worksheet.set_column("A:F", 18) # set column widths to 18


# TAB 2 - BIKESPACE REPORTS
worksheet = workbook.add_worksheet('DamageReports')
writer.sheets['DamageReports'] = worksheet

# write header content
worksheet.write(
  'A1', 
  "Damaged Bicycle Parking Reports", 
  bold,
)
(report_matches_unique
  .drop(columns=["geometry"])
  .to_excel(
    writer, 
    sheet_name='DamageReports', 
    startrow=2, 
    startcol=0,
    header=True,
))


# TAB 3 - MATCHED CITY BICYCLE PARKING FEATURES
worksheet = workbook.add_worksheet('CityFeatures')
writer.sheets['CityFeatures'] = worksheet
# write header content
worksheet.write(
  'A1', 
  "Matched City Bicycle Parking Features", 
  bold,
)
(matched_city_features_unique
  .drop(columns=["geometry"])
  .to_excel(
    writer, 
    sheet_name='CityFeatures', 
    startrow=2, 
    startcol=0,
    header=True,
))


workbook.close()
